In [7]:
import os
import sys
import random 
SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))
SPARK_UI_PORT = random.choice(range(10000, 10200))

In [ ]:

    
    
    #
    # Relabel
    #
    
    #
    # Нечто странное
    #
    train_flag = (data.where(F.col("label").isNull()).count() == 0)
    if not train_flag: df2 = df2.drop('label')
    

In [8]:
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import *
from pyspark.ml import Estimator, Transformer
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF
import pyspark.sql.functions as F

In [9]:
conf = SparkConf()
conf.set("spark.ui.port", SPARK_UI_PORT)
spark = SparkSession.builder.config(conf=conf).appName("HW5b").getOrCreate()
spark.sparkContext.setLogLevel('WARN')    

In [10]:
raw_data_path = '/datasets/amazon/all_reviews_5_core_train_extra_small_sentiment.json'
processed_data_path = 'HW5/processed.parquet'

In [11]:
schema = StructType([
        StructField("asin", StringType()),
        StructField("id", LongType()),
        StructField("label", IntegerType()),
        StructField("reviewText", StringType()),
        StructField("reviewTime", DateType()),
        StructField("reviewerID", StringType()),
        StructField("reviewerName", StringType()),
        StructField("vote", IntegerType()),
        StructField("summary", StringType()),
        StructField("unixReviewTime", TimestampType()),
        StructField("verified", BooleanType())
    ])

dataset = spark.read.json(raw_data_path, schema=schema,dateFormat='MM dd, yyyy').cache()

In [12]:
dataset.show(2, vertical=True)

-RECORD 0------------------------------
 asin           | B000VX4W78           
 id             | 72900                
 label          | 1                    
 reviewText     | Purchased these f... 
 reviewTime     | 2009-11-30           
 reviewerID     | A3BH3XJBBU7FF2       
 reviewerName   | The Dragon           
 vote           | null                 
 summary        | Works great, nice... 
 unixReviewTime | 2009-11-30 00:00:00  
 verified       | true                 
-RECORD 1------------------------------
 asin           | B0017U1KBK           
 id             | 104280               
 label          | 1                    
 reviewText     | This thing is too... 
 reviewTime     | 2017-01-04           
 reviewerID     | A408FUV9TO4EA        
 reviewerName   | Amber                
 vote           | null                 
 summary        | Five Stars           
 unixReviewTime | 2017-01-04 00:00:00  
 verified       | true                 
only showing top 2 rows



In [19]:
droper = SQLTransformer(statement="SELECT * FROM __THIS__ WHERE reviewText is not null")
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
hasher = HashingTF(numFeatures=100, binary=True, inputCol=tokenizer.getOutputCol(), outputCol="word_vector")
pipeline = Pipeline(stages=[
        droper,
        tokenizer,
        hasher        
    ])
processed_dataset = pipeline.fit(dataset).transform(dataset).select("id", "label", "word_vector")

In [20]:
processed_dataset.show(2, vertical=True)

-RECORD 0---------------------------
 id          | 72900                
 label       | 1                    
 word_vector | (100,[2,3,4,10,11... 
-RECORD 1---------------------------
 id          | 104280               
 label       | 1                    
 word_vector | (100,[16,34,44,46... 
only showing top 2 rows



In [21]:
def split_array_to_list(col):
    def to_list(v):
        return v.toArray().tolist()
    return F.udf(to_list, ArrayType(DoubleType()))(col)

processed_dataset = processed_dataset.select(["id", "label", split_array_to_list(F.col("word_vector")).alias("val")])\
        .select(["id", "label"] + [F.col("val")[i] for i in range(100)])

In [22]:
processed_dataset.show(2, vertical=True)

-RECORD 0---------
 id      | 72900  
 label   | 1      
 val[0]  | 0.0    
 val[1]  | 0.0    
 val[2]  | 1.0    
 val[3]  | 1.0    
 val[4]  | 1.0    
 val[5]  | 0.0    
 val[6]  | 0.0    
 val[7]  | 0.0    
 val[8]  | 0.0    
 val[9]  | 0.0    
 val[10] | 1.0    
 val[11] | 1.0    
 val[12] | 0.0    
 val[13] | 0.0    
 val[14] | 0.0    
 val[15] | 0.0    
 val[16] | 0.0    
 val[17] | 0.0    
 val[18] | 0.0    
 val[19] | 0.0    
 val[20] | 1.0    
 val[21] | 0.0    
 val[22] | 0.0    
 val[23] | 0.0    
 val[24] | 1.0    
 val[25] | 1.0    
 val[26] | 1.0    
 val[27] | 1.0    
 val[28] | 1.0    
 val[29] | 0.0    
 val[30] | 1.0    
 val[31] | 1.0    
 val[32] | 0.0    
 val[33] | 0.0    
 val[34] | 0.0    
 val[35] | 1.0    
 val[36] | 1.0    
 val[37] | 1.0    
 val[38] | 0.0    
 val[39] | 0.0    
 val[40] | 1.0    
 val[41] | 0.0    
 val[42] | 0.0    
 val[43] | 0.0    
 val[44] | 0.0    
 val[45] | 1.0    
 val[46] | 1.0    
 val[47] | 0.0    
 val[48] | 0.0    
 val[49] | 0

In [23]:
processed_data_path = 'HW5/processed_test.parquet'
processed_dataset.coalesce(1).write.parquet(processed_data_path,mode="overwrite")

In [ ]:
spark.stop()